# 作業 : (Kaggle)鐵達尼生存預測
https://www.kaggle.com/c/titanic

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# [作業目標]
- 試著模仿範例寫法, 在鐵達尼生存預測中, 觀察標籤編碼與獨編碼熱的影響

# [作業重點]
- 回答在範例中的觀察結果
- 觀察標籤編碼與獨熱編碼, 在特徵數量 / 邏輯斯迴歸分數 / 邏輯斯迴歸時間上, 分別有什麼影響 (In[3], Out[3], In[4], Out[4]) 

# 作業1
* 觀察範例，在房價預測中調整標籤編碼(Label Encoder) / 獨熱編碼 (One Hot Encoder) 方式，  
對於線性迴歸以及梯度提升樹兩種模型，何者影響比較大?

In [12]:
# 做完特徵工程前的所有準備 (與前範例相同)
import pandas as pd
import numpy as np
import copy, time
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder

data_path = '/content/drive/MyDrive/資料科學/第四屆基機器學習馬拉松/D30/'
df_train = pd.read_csv(data_path + 'titanic_train.csv')
df_test = pd.read_csv(data_path + 'titanic_test.csv')

train_Y = df_train['Survived']
ids = df_test['PassengerId']
df_train = df_train.drop(['PassengerId', 'Survived'] , axis=1)
df_test = df_test.drop(['PassengerId'] , axis=1)
df = pd.concat([df_train,df_test])
df.head()

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [13]:
#只取類別值 (object) 型欄位, 存於 object_features 中
object_features = []
for dtype, feature in zip(df.dtypes, df.columns):
    if dtype == 'object':
        object_features.append(feature)
print(f'{len(object_features)} Numeric Features : {object_features}\n')

# 只留類別型欄位
df = df[object_features]
df = df.fillna('None')
train_num = train_Y.shape[0]
df.head()

5 Numeric Features : ['Name', 'Sex', 'Ticket', 'Cabin', 'Embarked']



,Name,Sex,Ticket,Cabin,Embarked
0,"Braund, Mr. Owen Harris",male,A/5 21171,None,S
1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,PC 17599,C85,C
2,"Heikkinen, Miss. Laina",female,STON/O2. 3101282,None,S
3,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,113803,C123,S
4,"Allen, Mr. William Henry",male,373450,None,S


# 作業2
* 鐵達尼號例題中，標籤編碼 / 獨熱編碼又分別對預測結果有何影響? (Hint : 參考今日範例)

In [8]:
# 標籤編碼 + 羅吉斯迴歸

"""
Your Code Here
"""
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
for c in df.columns:
  df[c] = le.fit_transform(df[c])
train_X = df[:train_num]
start = time.time()

print('特徵總數:{}'.format(len(df.columns)))
print('shape:{}'.format(train_X.shape))
print('le_logistic_score:{}'.format(cross_val_score(LogisticRegression(), train_X, train_Y, cv=5).mean()))
print(f'time:{time.time()-start}sec')

特徵總數:5
shape:(891, 5)
le_logistic_score:0.7800138095537004
time:0.11979436874389648sec


In [16]:
# 獨熱編碼 + 羅吉斯迴歸

"""
Your Code Here
"""
# 先用get_dummies方法

df_temp = pd.get_dummies(df)
train_X = df_temp[:train_num]

start = time.time()

print('特徵總數:{}'.format(len(df_temp.columns)))
print('shape:{}'.format(train_X.shape))
print('one_hot_logistic_score:{}'.format(cross_val_score(LogisticRegression(), train_X, train_Y, cv=5).mean()))
print(f'time:{time.time()-start}sec')

特徵總數:2429
shape:(891, 2429)
one_hot_logistic_score:0.8013307388111229
time:0.7335913181304932sec


標籤編碼並不會增加特徵數量，且計算時間較為短，但其準確度卻比較低

獨熱編碼會跑出很多的特徵數，計算時間多了標籤編碼快七倍的時間，但其準確度卻比較高

In [19]:
# 標籤編碼 + GDBT
from sklearn.ensemble import GradientBoostingClassifier
df_temp = df.copy()

for c in df_temp.columns:
  df_temp[c] = le.fit_transform(df_temp[c])

start = time.time()
train_X = df_temp[:train_num]

print('特徵數量:{}'.format(len(train_X.columns)))
print('shape:{}'.format(train_X.shape))
print('le_GDBT_score:{}'.format(cross_val_score(GradientBoostingClassifier(), train_X, train_Y, cv=5).mean()))
print('time:{}sec'.format(time.time()-start))

特徵數量:5
shape:(891, 5)
le_GDBT_score:0.7946393823363255
time:0.6152610778808594sec


In [29]:
# 獨熱編碼 + GDBT
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.preprocessing import OneHotEncoder
df_temp = df.copy()
df_temp = pd.get_dummies(df_temp)

start = time.time()
train_X = df_temp[:train_num]

print('特徵數量:{}'.format(len(train_X.columns)))
print('shape:{}'.format(train_X.shape))
print('le_GDBT_score:{}'.format(cross_val_score(GradientBoostingClassifier(), train_X, train_Y, cv=5).mean()))
print('time:{}sec'.format(time.time()-start))

特徵數量:2429
shape:(891, 2429)
le_GDBT_score:0.8013181846713954
time:7.969715595245361sec
